# Define Classes for Field and Term Objects


In [86]:
import numpy as np

In [87]:
class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq
    
class term(object):
    def __init__(self, field_list, lorentz_contractions=[], spinor_contractions=[]):
        self.field_list = field_list #ordered list of field objects
        self.lorentz_contractions = lorentz_contractions
        self.spinor_contractions = spinor_contractions #list of 2-tuples, indicating index values of fields in self.fields that are contracted 
    def __eq__(self, other):
        eq = (self.field_list == other.field_list)&(self.lorentz_contractions == other.lorentz_contractions)&(self.spinor_contractions == other.spinor_contractions)
        return eq
    def __str__(self):
        return 'fields: ' + str([item.get_symbol() for item in self.field_list]) + '\n' \
            + 'contractions: ' + str(self.contractions)
    def get_field_list(self):
        return self.field_list
    def get_lorentz_contractions(self):
        return self.lorentz_contractions
    def get_spinor_contractions(self):
        return self.spinor_contractions
    def set_field_list(self, field_list):
        self.field_list = field_list
    def set_lorentz_contractions(self, lorentz_contractions):
        self.lorentz_contractions = lorentz_contractions
    def set_spinor_contractions(self, spinor_contractions):
        self.spinor_contractions = spinor_contractions    
    def massDim(self):
        massDim = 0
        field_list = self.field_list
        #print(field_list)
        for field in field_list:
            massDim += field.get_massDim()
        return massDim 
    def get_field_symbols(self):
        field_symbols = []
        for item in self.field_list:
            field_symbols.append(item.get_symbol())
        return field_symbols    

In [88]:
#spacetime derivative
D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)

#dynamical fields
F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)

#16 spinor space blinears: 1 lorentz scalar, 4 lorentz vectors, 6 lorentz tensors, 4 lorentz pseudovector, 1 lorentz pseudoscalar. 
#not dynamical fields, constant matrices in spinor space
_S_ = field('_S_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
_V_ = field('_V_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
_T_ = field('_T_', massDim=0, lorentz_rank=2, spinor_rank=1, spinor_rank_conj=1) #6 of these
_Vp_ = field('_Vp_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
_Sp_ = field('_Sp_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these

# Generate All Terms of a Given Mass Dimension - with One Free Lorentz Index (for IBP), and with No Free Indices

*Preliminary thoughts*: Fix mass dimension, M. Fields in the term must be such that mass dimensions add up to M. Note that Dirac bilinear terms have no mass dimension, so the mass dimension does not directly constrain the allowed number of these. But constant matrices relate different index spaces (e.g., lorentz, spinor). 
- count number of derivatives up to and including 0, ..., M-1 (only M-1 and not M because a term must contain at least one dynamical field). 
- for each number of derivatives, find all combinations of fields consistent with the mass dimension (forgetting about contractions for the moment).
- for each number of derivatives, and for each such combination of fields, perform all possible contractions, including all combinations of constant bilinear spinor space matrices. 
- define an equivalence relation between terms (based on commutation/anticommutation of derivatives/fields, various identities such as Fierz identities) to remove repeats. 

## Generate All Combinations of Fields with a Given Mass Dimension

In [89]:
def generate_field_combos_d(massDim, num_derivs):
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=1.5, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0) #Psi
    Pb = field('Pb', massDim=1.5, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1) #Psi_bar
    
    if massDim <= num_derivs:
        return []
    else:
        combo_root = num_derivs*[D]
        combo_roots_list = [combo_root]
        massDim_res_max = massDim - num_derivs
       
        while massDim_res_max >= 2:
            combo_roots_list_new = []

            #print("")
            #print("combo_roots_list:")
            #for combo in combo_roots_list:
                #print(str([item.get_symbol() for item in combo]))
        
            for i in range(len(combo_roots_list)):
                massDim_root = sum([item.get_massDim() for item in combo_roots_list[i]])
                massDim_res = massDim - massDim_root
                #append any elements that already have residual massDim less than 2
                if massDim_res < 2:
                    combo_roots_list_new.append(combo_roots_list[i])
                if massDim_res >= 2:
                    combo_root_new1 = combo_roots_list[i].copy()
                    combo_root_new1.append(F)
                    combo_roots_list_new.append(combo_root_new1)
                if massDim_res >= 3:
                    combo_root_new2 = combo_roots_list[i].copy()
                    combo_root_new2.append(Pb)
                    combo_root_new2.append(P)
                    combo_roots_list_new.append(combo_root_new2)  
            #for combo in combo_roots_list_new:
                #print(str([item.get_symbol() for item in combo]))
            
            massDim_res_max = max([(massDim - sum([item.get_massDim() for item in combo_roots_list_new[i]])) for i in range(len(combo_roots_list_new))])
            #print("massDim_res_max: " + str(massDim_res_max))
            combo_roots_list = combo_roots_list_new
        
        #prune combo_roots_list, removing any terms with mass dimension massDim-1
        #print("")
        #print("combo_roots_list_final:")
        #for combo in combo_roots_list:
            #print(str([item.get_symbol() for item in combo]))
            
        combos_list_pruned = []
        for combo in combo_roots_list:
            combo_massDim = sum([item.get_massDim() for item in combo])
            if combo_massDim == massDim:
                combos_list_pruned.append(combo)
            else:
                pass    

    return combos_list_pruned

In [90]:
field_combos = generate_field_combos_d(10, 4)
print("len(field_combos): " + str(len(field_combos)))
for combo in field_combos:
    print(str([item.get_symbol() for item in combo]))

len(field_combos): 2
['D', 'D', 'D', 'D', 'F', 'F', 'F']
['D', 'D', 'D', 'D', 'Pb', 'P', 'Pb', 'P']


In [91]:
def generate_field_combos(massDim):
    combos_list = []
    for num_derivs in range(massDim):
        combos_list_d = generate_field_combos_d(massDim, num_derivs)
        combos_list += combos_list_d
    return combos_list        

In [92]:
field_combos = generate_field_combos(10)
print("len(field_combos): " + str(len(field_combos)))
for combo in field_combos:
    print(str([item.get_symbol() for item in combo]))

len(field_combos): 26
['F', 'F', 'F', 'F', 'F']
['F', 'F', 'Pb', 'P', 'Pb', 'P']
['F', 'Pb', 'P', 'F', 'Pb', 'P']
['F', 'Pb', 'P', 'Pb', 'P', 'F']
['Pb', 'P', 'F', 'F', 'Pb', 'P']
['Pb', 'P', 'F', 'Pb', 'P', 'F']
['Pb', 'P', 'Pb', 'P', 'F', 'F']
['D', 'F', 'F', 'F', 'Pb', 'P']
['D', 'F', 'F', 'Pb', 'P', 'F']
['D', 'F', 'Pb', 'P', 'F', 'F']
['D', 'Pb', 'P', 'F', 'F', 'F']
['D', 'Pb', 'P', 'Pb', 'P', 'Pb', 'P']
['D', 'D', 'F', 'F', 'F', 'F']
['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P']
['D', 'D', 'Pb', 'P', 'F', 'Pb', 'P']
['D', 'D', 'Pb', 'P', 'Pb', 'P', 'F']
['D', 'D', 'D', 'F', 'F', 'Pb', 'P']
['D', 'D', 'D', 'F', 'Pb', 'P', 'F']
['D', 'D', 'D', 'Pb', 'P', 'F', 'F']
['D', 'D', 'D', 'D', 'F', 'F', 'F']
['D', 'D', 'D', 'D', 'Pb', 'P', 'Pb', 'P']
['D', 'D', 'D', 'D', 'D', 'F', 'Pb', 'P']
['D', 'D', 'D', 'D', 'D', 'Pb', 'P', 'F']
['D', 'D', 'D', 'D', 'D', 'D', 'F', 'F']
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'Pb', 'P']
['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'F']


In [93]:
def convert_to_symbol_list(field_list):
    symbol_list = [item.get_symbol() for item in field_list]
    return(symbol_list)

def convert_to_field_list(symbol_list):
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
    Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)
    
    fields = [D, F, P, Pb]
    
    field_list = [Field for symbol in symbol_list for Field in fields if symbol == Field.get_symbol()]
    
    return field_list
                
def field_counts(symbol_list):
    D_count = 0
    F_count = 0
    PPb_count = 0 #Pb always comes with P, so no need to count Pb
    for item in symbol_list:
        if item == 'D':
            D_count += 1
        elif item == 'F':
            F_count += 1
        elif item == 'P':
            PPb_count += 1 
    field_counts = [D_count, F_count, PPb_count]
    return field_counts

def combos_equiv(symbol_list1, symbol_list2):
    return field_counts(symbol_list1) == field_counts(symbol_list2)



In [94]:
symbol_list = ['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P']
[item.get_symbol() for item in convert_to_field_list(symbol_list)]

['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P']

In [95]:
field_counts(convert_to_symbol_list(field_combos[11]))

[1, 0, 3]

In [96]:
combos_equiv(convert_to_symbol_list(field_combos[10]), convert_to_symbol_list(field_combos[11]))

False

In [97]:
def reduce_field_combos(combo_list):
    #INPUT
    #combo_list: list of lists of field symbols (strings)
    #OUTPUT
    #combo_list_reduced: list of lists of field symbols, with repeat combos removed
    #EXPLANATION: removes repeat field combos from combo_list
    #print("len(combo_list): " + str(len(combo_list)))
    combo_list_reduced = [combo_list[0]]
    for i in range(len(combo_list)):
        #print("combo1: " + str(combo1))
        #print("")
        #print("i: " + str(i))
        #print("len(combo_list_reduced): " + str(len(combo_list_reduced)))
        for j in range(len(combo_list_reduced)):
            #print("combo2: " + str(combo2))
            #print("j: " + str(j))
            if combos_equiv(combo_list[i], combo_list_reduced[j]):
                #print("break")
                break
        else:
            combo_list_reduced.append(combo_list[i])
    return combo_list_reduced  
    
    

In [98]:
massDim = 10
symbol_lists = []
for combo in generate_field_combos(massDim):
    symbol_lists.append(convert_to_symbol_list(combo))
symbol_lists
    
combo_list_reduced = reduce_field_combos(symbol_lists)
len(combo_list_reduced)
combo_list_reduced

[['F', 'F', 'F', 'F', 'F'],
 ['F', 'F', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'F', 'F', 'F', 'Pb', 'P'],
 ['D', 'Pb', 'P', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'D', 'F', 'F', 'F', 'F'],
 ['D', 'D', 'F', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'D', 'D', 'F', 'F', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'F', 'F', 'F'],
 ['D', 'D', 'D', 'D', 'Pb', 'P', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'F', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'F', 'F'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'F']]

## Generate Derivative Assignments for Each Combination of Fields

In [99]:
#helper functions for generate_derivative_assignments
def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def sums_to_n(n, m): # generates partitions of derivatives to each field
    #INPUT
    #n: number we want to sum to; i.e. total number of derivatives
    #m: number of digits in sum; i.e. total number of fields
    #OUTPUT
    #list of lists of digits, with each sublist no longer than m
    #EXPLANATION
    #list of combos for n can be generated from list of combos for n-1 either by appending 1 to each of the combos
    #for n-1, or adding 1 to just one of the elements in each combo. To ensure no repeats, we require the lists to be
    #non-increasing.
    combos_list = []
    if n==0:
        combo = m*[0]
        combos_list.append(combo)
        return combos_list
    elif n==1:
        combo = [1] + (m-1)*[0]
        combos_list.append(combo)
        return combos_list
    elif m==0:
        return []
    else:
        combos_list_prev = sums_to_n(n-1, n-1)
        for combo in combos_list_prev:
            combo_new1 = combo + [1]
            if non_increasing(combo_new1) and combo_new1 not in combos_list:
                combos_list.append(combo_new1) 
            for i in range(len(combo)):
                combo_new2 = combo.copy()
                combo_new2[i] += 1
                if non_increasing(combo_new2) and combo_new2 not in combos_list:
                    combos_list.append(combo_new2) 
        combos_list_trunc = [combo for combo in combos_list if len(combo) <= m]    
        return combos_list_trunc

def pad_w_zeros(list_of_lists, size):
    #INPUT
    #list_of_lists: list of lists of numbers; all sublists should have length less than or equal to size
    #size: int, common size of all lists after padding
    #OUTPUT
    #list_of_lists_padded: padded list of lists, where each sublist has length size
    
    if size < max([len(sublist) for sublist in list_of_lists]):
        print("'size' input must be larger than or equal max length of sublists in 'list_of_lists' input.")
        return None
    else:
        list_of_lists_padded = []
        for sublist in list_of_lists:
            num_zeros = size - len(sublist)
            sublist_padded = sublist + num_zeros*[0]
            list_of_lists_padded.append(sublist_padded)
        return list_of_lists_padded
        
def sums_to_n_padded(n, m):
    return pad_w_zeros(sums_to_n(n, m), m) 

def generate_derivative_assignments_recursively(field_symbol_list, num_fields_list, d):
    #INPUT
    #field_symbol_list: list of field symbols - e.g., ['F', 'Pb', 'P']
    #num_fields_list: number of each type of non-derivative field [num_F, num_Pb, num_P], must be same length
    #as field_symbol_list
    #d: total number of derivatives acting on all fields
    #OUTPUT
    #derivative_assignments_list: list of lists, with each sublist indicating the number of derivatives assigned 
    #to each (non-derivative) field. number of derivatives preceding each field indicates number of derivatives
    #acting on that field: e.g. [['D', 'D', 'F', 'D', 'F', 'D', 'D', 'D', 'Pb', 'D', 'P', 'D', 'D', 'P'],
    #['D', 'F', 'D', 'Pb', 'D', 'D', 'D', 'Pb', 'D', 'D', 'P'], ...]
    
    #find index of last non-zero term in num_fields_list. this will serve as a base case for recursion
    last_nonzero_index = None #initialize last_nonzero_index 
    for i in range(1, len(num_fields_list)+1): #find index (from end of list) of last non-zero term in num_fields_list; deposit all 
        #remaining derivatives with this field type.
        if num_fields_list[-i]!=0:
            last_nonzero_index = -i
            break
    #print("last_nonzero_index: " + str(last_nonzero_index))
    if len(field_symbol_list)!=len(num_fields_list):
        print("field_symbol_list and num_fields_list must have same length.")
        return []
    if num_fields_list==[] or field_symbol_list==[]:
        print("num_fields_list and field_symbol_list cannot be empty.")
        return []
    if d==0:
        fields = []
        for i in range(len(field_symbol_list)):
            field_symbol = field_symbol_list[i]
            for j in range(num_fields_list[i]):
                fields.append(field_symbol)
        return [fields]
    elif num_fields_list[0]==0: #if no fields of a particular type, move on to next type
        return generate_derivative_assignments_recursively(field_symbol_list[1:], num_fields_list[1:], d)
    elif len(num_fields_list)==-last_nonzero_index: #if we are on the last field type with non-zero entry in 
        #print("last non-zero entry in num_fields_list")
        n_0 = num_fields_list[0]
        field_symbol = field_symbol_list[0]
        derivative_assignments_list = []
        derivs_0 = sums_to_n_padded(d, n_0)
        appended_list = []
        for num_derivs_list in derivs_0:
            appended = []
            for j in num_derivs_list:
                appended += j*['D']
                appended += [field_symbol]
            appended_list.append(appended)
        return appended_list
    else:
        n_0 = num_fields_list[0]
        field_symbol = field_symbol_list[0]
        derivative_assignments_list = []
        #for each possible number of derivatives acting on field species corresponding to 0th index of num_fields_list,
        #compute all possible ways of distributing derivatives among those fields. 
        for d_0 in range(d+1):
            derivs_0 = sums_to_n_padded(d_0, n_0)
            #generate a list of sublists of symbols associated with each element of derivs_0 - e.g., one such  
            #sublist might be ['D', 'D', 'F', 'D', 'F', 'D', 'D', 'D', 'F']
            appended_list = []
            for num_derivs_list in derivs_0:
                appended = []
                for j in num_derivs_list:
                    appended += j*['D']
                    appended += [field_symbol]
                    #print("field_symbol: " + str(field_symbol))
                    #print("appended: " + str(appended))
                appended_list.append(appended)
            #print("field_symbol_list[1:]: " + str(field_symbol_list[1:]))
            derivative_assignments_list_old = generate_derivative_assignments_recursively(field_symbol_list[1:], num_fields_list[1:], d-d_0)
            for appended in appended_list:
                for derivative_assignment_old in derivative_assignments_list_old:
                    derivative_assignment_new = appended + derivative_assignment_old
                    derivative_assignments_list.append(derivative_assignment_new)
        return derivative_assignments_list

    
def generate_derivative_assignments(field_combo):
    #INPUT
    #field_combo: List of string symbols for derivatives and non-derivative fields (called simply fields here).  
    #OUTPUT
    #derivative_assigments: list of lists, where each sublist contains a distinct assignment of derivatives in
    #field_combo to fields in field_combo
    #EXPLANATION: generates all assignments of derivatives to fields for a given combination. 
    d = 0 #initialize number of derivatives
    n_F = 0 #initialize number of F fields
    n_P = 0 #initialize number of P fields
    derivative_assigment_list = [] #lists of lists, where the elements of each sublist are field symbols; the 
    #number of 'D' symbols preceding a given non-derivative field indicates the number of derivatives acting on 
    #that field.
    for item in field_combo:
        if item == 'D':
            d += 1
        elif item == 'F':
            n_F += 1
        elif item == 'P':
            n_P += 1
        #no need to count Pb since there are the same number of these as of P
    field_symbol_list = ['F', 'Pb', 'P']
    num_fields_list = [n_F, n_P, n_P]
    return generate_derivative_assignments_recursively(field_symbol_list, num_fields_list, d)


In [100]:
field_symbol_list = ['F', 'Pb', 'P']
num_fields_list = [1, 5, 5] 
d = 4 
generate_derivative_assignments_recursively(field_symbol_list, num_fields_list, d)

[['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'P',
  'D',
  'D',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'D',
  'D',
  'P',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'P',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'D',
  'D',
  'P',
  'P',
  'P',
  'P',
  'P'],
 ['F',
  'D',
  'Pb',
  'D',
  'Pb',
  'Pb',
  'Pb',
  'Pb',
  'D',
  'P',
  'D',
  'P',
  'P',
  'P',
  'P'],
 

In [101]:
field_combo = ['D', 'D', 'D', 'F', 'F', 'Pb', 'P']
generate_derivative_assignments(field_combo)

[['F', 'F', 'Pb', 'D', 'D', 'D', 'P'],
 ['F', 'F', 'D', 'Pb', 'D', 'D', 'P'],
 ['F', 'F', 'D', 'D', 'Pb', 'D', 'P'],
 ['F', 'F', 'D', 'D', 'D', 'Pb', 'P'],
 ['D', 'F', 'F', 'Pb', 'D', 'D', 'P'],
 ['D', 'F', 'F', 'D', 'Pb', 'D', 'P'],
 ['D', 'F', 'F', 'D', 'D', 'Pb', 'P'],
 ['D', 'F', 'D', 'F', 'Pb', 'D', 'P'],
 ['D', 'F', 'D', 'F', 'D', 'Pb', 'P'],
 ['D', 'D', 'F', 'F', 'Pb', 'D', 'P'],
 ['D', 'D', 'F', 'F', 'D', 'Pb', 'P'],
 ['D', 'D', 'F', 'D', 'F', 'Pb', 'P'],
 ['D', 'D', 'D', 'F', 'F', 'Pb', 'P']]

## Generate All Full and One-Free-Lorentz-Index Contractions from a Given Derivative Assignment

### Contracting Spinor Indices

Consider a term of the form

$(D_{\mu^{1}_{1}} ... D_{\mu^{1}_{k_{1}}} \bar{\psi}) ... (D_{\mu^{n}_{1}} ... D_{\mu^{n}_{k_{n}}} \bar{\psi}) (D_{\nu^{1}_{1}} ... D_{\nu^{1}_{l_{1}}} \psi) ... (D_{\nu^{n}_{1}} ... D_{\nu^{n}_{l_{n}}} \psi) (D_{\alpha_{1}} ... D_{\alpha_{p}} F_{\kappa \gamma})$.

There are the same number of $\bar{\psi}$ as $\psi$. Any (D... $\bar{\psi}$) can be contracted in spinor space with any  (D... $\psi$). There are $n$ of the first and $n$ of the second, so $n^2$ altogether. For each distinct $A_{i} = D_{\mu^{i}_{1}} ... D_{\mu^{i}_{k_{1}}} \bar{\psi}$ and $B_{j} = D_{\nu^{j}_{1}} ... D_{\nu^{j}_{l_{j}}} \psi$, we have 

$A_{i} M B_{j}$, 

where $M = \{I, \gamma_{\mu}, \sigma_{\mu \nu}, \gamma_{5}\gamma_{\mu}, \gamma_{5} \}$. Thus, we have 5 possibilities for each pair of $\bar{\psi}$ and $\psi$. For each of the $n^2$ terms, we have $5^{n}$ distinct possibilities. So, including the different matrices in $M$, we generate

$n^{2}5^{n}$

terms. Write a function, *generate_spinor_contractions(derivative_assignment)*, which takes a derivative assignment and returns all such $n^{2}5^{n}$ terms. 





In [102]:
'''
def generate_spinor_contractions_from_indices(Pb_indices, P_indices):
    #Check that lists are of same length
    if len(Pb_indices) != len(P_indices):
        print('Pb_indices, P_indices must be of same length.')
        return None
    elif len(Pb_indices)==1: #base case
        list_of_contraction_lists = [[(Pb_indices[0], P_indices[0])]]
        return list_of_contraction_lists
    else:
        n = len(Pb_indices)
        list_of_contraction_lists = [] #store lists of contractions of Pb with P; each of length n
        for i in range(n):
            for j in range(n):
                contraction = (Pb_indices[i], P_indices[j])
                Pb_indices_copy = Pb_indices.copy()
                P_indices_copy = P_indices.copy()
                
                del(Pb_indices_copy[i])
                del(P_indices_copy[j])

                for contraction_list in generate_spinor_contractions_from_indices(Pb_indices_copy, P_indices_copy):
                    contraction_list_new = [contraction] + contraction_list
                    list_of_contraction_lists.append(contraction_list_new)          
                
    return list_of_contraction_lists
    


def reduce_spinor_contrations(list_of_contraction_lists):
    unique = []
    set_list = [set(item) for item in list_of_contraction_lists]
    for item in set_list:
        if set(item) not in unique:
            unique.append(item)
    return unique

        
'''

def generate_spinor_contractions_from_indices(Pb_indices, P_indices):
    #Check that lists are of same length
    if len(Pb_indices) != len(P_indices):
        print('Pb_indices, P_indices must be of same length.')
        return None
    elif len(Pb_indices)==1: #base case
        list_of_contraction_lists = [[(Pb_indices[0], P_indices[0])]]
        return list_of_contraction_lists
    else:
        n = len(Pb_indices)
        list_of_contraction_lists = [] #store lists of contractions of Pb with P; each of length n
        for j in range(n):
            contraction = (Pb_indices[0], P_indices[j]) #start with first index for Pb_indices; as it is deleted,
            #the recursion will carry us through each element of Pb_indices and cover all pairs of Pb and P just once. 
            Pb_indices_copy = Pb_indices.copy()
            P_indices_copy = P_indices.copy()

            del(Pb_indices_copy[0])
            del(P_indices_copy[j])

            for contraction_list in generate_spinor_contractions_from_indices(Pb_indices_copy, P_indices_copy):
                contraction_list_new = [contraction] + contraction_list
                list_of_contraction_lists.append(contraction_list_new)          

    return list_of_contraction_lists

def generate_spinor_contractions(derivative_assignment):
    #separate out terms of the form D...DP and D...DPb, or P, Pb. 
    Pb_indices = [] #collect Pb's, Pb derivatives
    P_indices = [] #collect P's, P derivatives
    spinor_contractions = [] #list of 2-tuples
    #find indices with Pb or P
    for i in range(len(derivative_assignment)):
        if derivative_assignment[i]=='Pb':
            Pb_indices.append(i)
        if derivative_assignment[i]=='P':
            P_indices.append(i)
    
    list_of_contraction_lists = generate_spinor_contractions_from_indices(Pb_indices, P_indices)
        
    return list_of_contraction_lists


In [103]:
#derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'P', 'P']
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
spinor_contractions = generate_spinor_contractions(derivative_assignment)
spinor_contractions

[[(1, 6), (2, 8), (3, 10)],
 [(1, 6), (2, 10), (3, 8)],
 [(1, 8), (2, 6), (3, 10)],
 [(1, 8), (2, 10), (3, 6)],
 [(1, 10), (2, 6), (3, 8)],
 [(1, 10), (2, 8), (3, 6)]]

For each contraction of a $\bar{\Psi}$ with a $\Psi$, we can insert any of the bilinears $M = \{I, \gamma_{\mu}, \sigma_{\mu \nu}, \gamma_{5}\gamma_{\mu}, \gamma_{5} \}$ to get $\left(D_{\mu_{1}} ... D_{\mu_{k}}\bar{\Psi} \right)\left(M\right) \left(D_{\nu_{1}} ... D_{\nu_{l}}\Psi \right)$. For each 

In [104]:
'''
def generate_bilinear_contractions_from_single_list(derivative_assignment):
    M = ['_S_', '_V_', '_T_', '_Vp_', '_Sp_']
    list_of_contraction_lists = generate_spinor_contractions(derivative_assignment)
    list_of_bilinear_contraction_lists = []
    for contraction_list in list_of_contraction_lists:
        bilinear_contraction_list = []
        for contraction in contraction_list:
            for m in M:
                contraction_bilinear = (contraction, m]
                bilinear_contraction_list.append(contraction_bilinear)
        list_of_bilinear_contraction_lists.append(bilinear_contraction_list)
    return list_of_bilinear_contraction_lists
'''

def generate_bilinear_contractions_from_single_list(contraction_list):
    #bilinears
    M = ['_S_', '_V_', '_T_', '_Vp_', '_Sp_']
    #base case
    if len(contraction_list)==1:
        list_of_bilinear_contraction_lists = []
        for m in M:
            contraction_list_new =  [(contraction_list[0], m)]
            list_of_bilinear_contraction_lists.append(contraction_list_new)
        return list_of_bilinear_contraction_lists
    else:
        list_of_bilinear_contraction_lists = []
        for m in M:
            bilinear_contraction = (contraction_list[0], m)
            for contraction_list_trunc in generate_bilinear_contractions_from_single_list(contraction_list[1:]):
                contraction_list_new = [bilinear_contraction] + contraction_list_trunc
                list_of_bilinear_contraction_lists.append(contraction_list_new)
        return list_of_bilinear_contraction_lists
    
def generate_bilinear_contractions(derivative_assignment):
    list_of_contraction_lists = generate_spinor_contractions(derivative_assignment)
    list_of_bilinear_contraction_lists = []
    for contraction_list in list_of_contraction_lists:
        list_of_bilinear_contraction_lists += generate_bilinear_contractions_from_single_list(contraction_list)
    return list_of_bilinear_contraction_lists
    
 

In [105]:
contraction_list = [(1, 6), (2, 8), (3, 10)]
generate_bilinear_contractions_from_single_list(contraction_list)

[[((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_T_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_Vp_')],
 [((1, 6), '_S_'), ((2, 8), '_S_'), ((3, 10), '_Sp_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_T_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_Vp_')],
 [((1, 6), '_S_'), ((2, 8), '_V_'), ((3, 10), '_Sp_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_T_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_Vp_')],
 [((1, 6), '_S_'), ((2, 8), '_T_'), ((3, 10), '_Sp_')],
 [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_S_')],
 [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_V_')],
 [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_T_')],
 

In [106]:
len(generate_bilinear_contractions(derivative_assignment))

750

### Remove Redundant Spinor Contractions

Next, remove redundant spinor contractions. Two spinor contractions are equivalent iff 

- they have the same number $n$ of $\bar{\psi}$ and $\psi$. 
- they have the same set $\{(d_{1}, d_{2}, m)\}$ of $n$ triplets, where $d_{1}$ is the number of derivatives acting on $\bar{\psi}$, $d_{2}$ is the number of derivatives acting on $\psi$, and $m \in M$, or each pair of $\bar{\psi}$ and $\psi$.

To do this, first write a function *equiv_spinor_contractions(spinor_contraction1, spinor_contraction2)* that takes two spinor contractions (with all Lorentz indices uncontracted), and outputs True if they are equivalent and False if not. Then write a function *reduce_spinor_contraction_list(spinor_contraction_list)* that takes a list of spinor contractions and returns that list with repetitions removed. 

In [107]:
def spinor_contractions_equiv(spinor_contraction1, spinor_contraction2):
    #INPUT
    #-spinor_contraction1: a list with 2 elements - 1) a list of field symbols indicating derivative assignments to different 
    #fields, assumed ordered with F's to the left of Pb's to the left of P's, 
    #2) a list of tuples indicating the locations of contracted Pb and P in the derivative assignment list,
    #as well as the bilinear matrix with which they are contracted. 
    #-spinor_contraction2: same as spinor_contraction1
    #OUTPUT
    #equiv: True if the spinor contractions are equivalent and False otherwise. 
    #EXPLANATION
    #once it has been checked that the derivative assignments are the same, for each Pb/P contraction, identify a 
    #tuple of 1) the bilinear matrix, 2) the number of derivatives acting on Pb, 3) the number of derivatives acting 
    #on P. for spinor_contraction1 and spinor_contraction2 to be equivalent, the set of such tuples for both should
    #be the same. 
    
    #check that derivative assignment lists are the same
    derivative_assignment1 = spinor_contraction1[0]
    derivative_assignment2 = spinor_contraction2[0]
    
    if derivative_assignment1 != derivative_assignment2:
        return False 
    
    #check that the numbers of Pb/P contractions are the same
    bilinear_contraction_list1 = spinor_contraction1[1]
    bilinear_contraction_list2 = spinor_contraction2[1]
    
    if len(bilinear_contraction_list1) != len(bilinear_contraction_list2):
        return False
    
    #Extract set of (n_d, n_db, M) triples for spinor_contraction1 and spinor_contraction2
    triples_list1 = extract_derivative_bilinear_triples(spinor_contraction1)
    triples_list2 = extract_derivative_bilinear_triples(spinor_contraction2)
    
    if set(triples_list1) == set(triples_list2):
        equiv = True
    else:
        equiv = False

    return equiv


def extract_derivative_bilinear_triples(spinor_contraction):
    #INPUT
    #-spinor_contraction: a list with 2 elements - 1) a list of field symbols indicating derivative assignments to different 
    #fields, assumed ordered with F's to the left of Pb's to the left of P's, 
    #2) a list of tuples indicating the locations of contracted Pb and P in the derivative assignment list,
    #as well as the bilinear matrix with which they are contracted. 
    #OUTPUT
    #triples_list: a list of 3-tuples (n_db, n_d, M), one for each Pb/P pair in spinor_contraction. for each 
    #Pb/P pair, the corresponding 3-tuple indicates the number n_db of derivatives acting on the Pb, n_d the
    #number of derivatives acting on the P, and M the bilinear matrix sandwiched between them. 
    
    #extract derivative assignment list and bilinear contraction list
    derivative_assignment = spinor_contraction[0]
    bilinear_contraction_list = spinor_contraction[1]
    
    triples_list = [] #store set of triples (n_dp, n_d, M), one for each Pb/P contraction. 
    #count number of derivatives n_dp and n_p acting on each Pb and P, respectively. 
    for i in range(len(bilinear_contraction_list)):
        #unpack bilinear contraction
        bilinear_contraction = bilinear_contraction_list[i]
        contraction_indices = bilinear_contraction[0]
        M = bilinear_contraction[1]
        P_index = contraction_indices[1]
        Pb_index = contraction_indices[0]
        
        #count number of D's acting on field P at P_index in derivative_assignment
        n_d = 0
        i = 1
        while P_index - i >= 0 and derivative_assignment[P_index - i] == 'D':
            n_d += 1
            i += 1
            
        #count number of D's acting on field Pb at Pb_index in derivative_assignment
        n_db = 0
        i = 1
        while Pb_index - i >= 0 and derivative_assignment[Pb_index - i] == 'D':
            n_db += 1
            i += 1
        
        triple = (n_db, n_d, M)
        triples_list.append(triple)
    
    return triples_list
    

In [108]:
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
bilinear_contraction_list1 = [((1, 6), '_V_'), ((2, 8), '_V_'), ((3, 10), '_T_')]
bilinear_contraction_list2 = [((1, 6), '_V_'), ((2, 10), '_V_'), ((3, 8), '_T_')]
#bilinear_contraction_list2 = [((1, 6), '_S_'), ((2, 8), '_Vp_'), ((3, 10), '_S_')]

spinor_contraction1 = [derivative_assignment, bilinear_contraction_list1]
spinor_contraction2 = [derivative_assignment, bilinear_contraction_list2]

print(spinor_contractions_equiv(spinor_contraction1, spinor_contraction2))

print(extract_derivative_bilinear_triples(spinor_contraction1))
print(extract_derivative_bilinear_triples(spinor_contraction2))

True
[(0, 2, '_V_'), (0, 1, '_V_'), (0, 1, '_T_')]
[(0, 2, '_V_'), (0, 1, '_V_'), (0, 1, '_T_')]


In [109]:
def reduced_spinor_contractions(derivative_assignment):
    #INPUT
    #derivative_assignment: list of field symbols where the number of 'D' preceding a field indicates the number
    #of derivatives acting on that field
    #OUTPUT
    #reduced_list: list of inequivalent bilinear spinor contractions associated with a particular derivative assignment
    
    #generate all bilinear spinor contractions
    list_of_bilinear_contractions = generate_bilinear_contractions(derivative_assignment)
    #initialize empty list for storing unique elements
    reduced_list = []
    
    for bilinear_contraction in list_of_bilinear_contractions:
        include_in_reduced_list = True
        #print("bilinear_contraction: " + str(bilinear_contraction))
        for item in reduced_list: #could make more efficient here; shouldn't need to loop through all elements of reduced_list for each bilinear contraction
            spinor_contraction1 = [derivative_assignment, bilinear_contraction]
            spinor_contraction2 = [derivative_assignment, item]
            if spinor_contractions_equiv(spinor_contraction1, spinor_contraction2):
                #print("equivalent item in reduced_list: " + str(item))
                include_in_reduced_list = False
                break
        if include_in_reduced_list:
            #print("append: " + str(bilinear_contraction))
            reduced_list.append(bilinear_contraction)
        
    return reduced_list

In [110]:
derivative_assignment = ['F', 'D', 'D', 'Pb', 'D', 'D', 'D', 'Pb', 'P', 'D', 'P']
reduced_list = reduced_spinor_contractions(derivative_assignment)
full_list = generate_bilinear_contractions(derivative_assignment)
for item in reduced_list:
    print(item)
'''
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
reduced_list = reduce_spinor_contractions(derivative_assignment)
full_list = generate_bilinear_contractions(derivative_assignment)
for item in reduced_list:
    print(item)
'''

[((3, 8), '_S_'), ((7, 10), '_S_')]
[((3, 8), '_S_'), ((7, 10), '_V_')]
[((3, 8), '_S_'), ((7, 10), '_T_')]
[((3, 8), '_S_'), ((7, 10), '_Vp_')]
[((3, 8), '_S_'), ((7, 10), '_Sp_')]
[((3, 8), '_V_'), ((7, 10), '_S_')]
[((3, 8), '_V_'), ((7, 10), '_V_')]
[((3, 8), '_V_'), ((7, 10), '_T_')]
[((3, 8), '_V_'), ((7, 10), '_Vp_')]
[((3, 8), '_V_'), ((7, 10), '_Sp_')]
[((3, 8), '_T_'), ((7, 10), '_S_')]
[((3, 8), '_T_'), ((7, 10), '_V_')]
[((3, 8), '_T_'), ((7, 10), '_T_')]
[((3, 8), '_T_'), ((7, 10), '_Vp_')]
[((3, 8), '_T_'), ((7, 10), '_Sp_')]
[((3, 8), '_Vp_'), ((7, 10), '_S_')]
[((3, 8), '_Vp_'), ((7, 10), '_V_')]
[((3, 8), '_Vp_'), ((7, 10), '_T_')]
[((3, 8), '_Vp_'), ((7, 10), '_Vp_')]
[((3, 8), '_Vp_'), ((7, 10), '_Sp_')]
[((3, 8), '_Sp_'), ((7, 10), '_S_')]
[((3, 8), '_Sp_'), ((7, 10), '_V_')]
[((3, 8), '_Sp_'), ((7, 10), '_T_')]
[((3, 8), '_Sp_'), ((7, 10), '_Vp_')]
[((3, 8), '_Sp_'), ((7, 10), '_Sp_')]
[((3, 10), '_S_'), ((7, 8), '_S_')]
[((3, 10), '_S_'), ((7, 8), '_V_')]
[((3, 10

"\nderivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']\nreduced_list = reduce_spinor_contractions(derivative_assignment)\nfull_list = generate_bilinear_contractions(derivative_assignment)\nfor item in reduced_list:\n    print(item)\n"

### Contracting Lorentz Indices

Next, for each spinor contraction, generate all possible contractions of Lorentz indices. For an even number of Lorentz indices, generate all ways of separating them into pairs. 

For an odd number of Lorentz indices, each index may serve as the free index; for each such choice of the free index, generate all possible groupings into pairs of the remaining even number of Lorentz indices; these one-Lorentz-index terms will serve to generate the integration by parts (IBP) relations. 

In [111]:
'''
def generate_fully_contracted_set(derivative_assignment):
    reduced_spinor_contractions = reduced_spinor_contractions(derivative_assignment)
    lorentz_scalar_terms = []
    lorentz_vector_terms = []
    
    return lorentz_scalar_terms, lorentz_vector_terms

def generate_fully_contracted(derivative_assignment, bilinear_contraction_list):
    #EXPLANATION
    #returns the full set of lorentz contractions of the free lorentz indices in the given spinor contraction
    lorentz_scalar_terms = []
    lorentz_vector_terms = []
    
    #convert all symbols in derivative_assignment, and all bilinears in blinear_contraction_list to field objects. 
    
    
    return lorentz_scalar_terms, lorentz_vector_terms
'''

def convert_to_term_object(derivative_assignment, bilinear_contraction_list):
    #INPUT
    #-derivative_assignment: list of symbols (D, F, P, Pb) denoting field objects. number of D's immediately preceding
    #an F, P, or Pb designates the number of derivatives acting on that field. 
    #-bilinear_contraction_list: list of tuples ((i, j), M) indicating contractions of Pb's with P's, and the bilinear
    #matrix M sandwiched between them. 
    #OUTPUT
    #-term_object: a term object with field list and spinor contractions dictated by inputs, but lorentz 
    #contractions unspecified.
    
    #construct field symbol list with bilinears inserted
    derivative_assignment_shifted = derivative_assignment.copy() #for insertion of bilinears
    shift = 0 #indices are shifted when we insert bilinears into field symbol list. keep track of shift. 
    for i in range(len(derivative_assignment)):
        if derivative_assignment[i]=='Pb':
            #move D...DP with which Pb is contracted to the immediate right of Pb
            M = None
            for j in range(len(bilinear_contraction_list)):
                if bilinear_contraction_list[j][0][0]==i:
                    M = bilinear_contraction_list[j][1]
                    shift += 1
                    break
            derivative_assignment_shifted = derivative_assignment_shifted[:i+shift] + [M] + derivative_assignment_shifted[i+shift:]
    
    '''
    #determine indices in derivative_assignment_shifted where bilinears have been inserted
    M_set = ['_S_', '_V_', '_T_', '_Vp_', '_Sp_']
    M_indices = []
    for i in range(len(derivative_assignment_shifted)):
        if derivative_assignment_shifted[i] in M_set:
            M_indices.append(int(i))
    '''
    #determine indices in derivative_assignment where bilinears are inserted, i.e., where there are Pb's
    Pb_indices = []
    for i in range(len(derivative_assignment)):
        if derivative_assignment[i]=='Pb':
            Pb_indices.append(int(i))
    
    #extract spinor contractions from bilinear_contraction_list for adjustment of contraction indices after 
    #insertion of bilinears
    spinor_contractions = []
    for item in bilinear_contraction_list:
        #print('type(item[0][0]): ' + str(type(item[0])))
        spinor_contractions += [item[0]]
    #print('spinor_contractions: ' + str(spinor_contractions))
    
    #fill in shifted list of spinor contractions
    spinor_contractions_shifted = []
    for contraction in spinor_contractions:
        contraction_new = list(contraction)
        for j in range(len(Pb_indices)):
            if Pb_indices[j] < contraction[0]:
                #print('Pb_indices[j] < contraction[0]; j=' + str(j))
                contraction_new[0] = contraction[0] + j + 1
            if Pb_indices[j] < contraction[1]:
                #print('Pb_indices[j] < contraction[1]; j=' + str(j))
                contraction_new[1] = contraction[1] + j + 1
        #split contraction: Pb contracts with bilinear one step to its right, and bilinear contracts with P. 
        contraction_new_split = [(contraction_new[0], contraction_new[0]+1), (contraction_new[0]+1, contraction_new[1])]
        spinor_contractions_shifted += contraction_new_split
        #spinor_contractions_shifted.append(tuple(contraction_new))
    #print('spinor_contractions_shifted: ' + str(spinor_contractions_shifted))
    
    #convert derivative_assignment_shifted to list of field objects
    D = field('D', massDim=1, lorentz_rank=1, spinor_rank=0, spinor_rank_conj=0)
    F = field('F', massDim=2, lorentz_rank=2, spinor_rank=0, spinor_rank_conj=0)
    P = field('P', massDim=3/2, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=0)
    Pb = field('Pb', massDim=3/2, lorentz_rank=0, spinor_rank=0, spinor_rank_conj=1)
    _S_ = field('_S_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
    _V_ = field('_V_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
    _T_ = field('_T_', massDim=0, lorentz_rank=2, spinor_rank=1, spinor_rank_conj=1) #6 of these
    _Vp_ = field('_Vp_', massDim=0, lorentz_rank=1, spinor_rank=1, spinor_rank_conj=1) #4 of these
    _Sp_ = field('_Sp_', massDim=0, lorentz_rank=0, spinor_rank=1, spinor_rank_conj=1) #1 of these
    
    field_list = []
    for symbol in derivative_assignment_shifted:
        if symbol=='D':
            field_list.append(D)
        elif symbol=='F':
            field_list.append(F)
        elif symbol=='P':
            field_list.append(P)
        elif symbol=='Pb':
            field_list.append(Pb)
        elif symbol=='_S_':
            field_list.append(_S_)
        elif symbol=='_V_':
            field_list.append(_V_)
        elif symbol=='_T_':
            field_list.append(_T_)
        elif symbol=='_Vp_':
            field_list.append(_Vp_)
        elif symbol=='_Sp_':
            field_list.append(_Sp_)
    
    #instantiate term object
    term_object = term(field_list=field_list, lorentz_contractions=[], spinor_contractions=spinor_contractions_shifted) 
             
    return term_object
    

In [112]:
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
bilinear_contraction_list = [((1, 6), '_V_'), ((2, 8), '_V_'), ((3, 10), '_T_')]
term_object = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
print(term_object.get_field_symbols())
print(term_object.get_spinor_contractions())

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]


In [113]:
derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'D', 'P', 'P']
#derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D','P']
spinor_contractions = generate_spinor_contractions(derivative_assignment)
print(spinor_contractions)
reduced_list = reduced_spinor_contractions(derivative_assignment)
for item in reduced_list:
    print(item)
    


[[(5, 8), (6, 9)], [(5, 9), (6, 8)]]
[((5, 8), '_S_'), ((6, 9), '_S_')]
[((5, 8), '_S_'), ((6, 9), '_V_')]
[((5, 8), '_S_'), ((6, 9), '_T_')]
[((5, 8), '_S_'), ((6, 9), '_Vp_')]
[((5, 8), '_S_'), ((6, 9), '_Sp_')]
[((5, 8), '_V_'), ((6, 9), '_S_')]
[((5, 8), '_V_'), ((6, 9), '_V_')]
[((5, 8), '_V_'), ((6, 9), '_T_')]
[((5, 8), '_V_'), ((6, 9), '_Vp_')]
[((5, 8), '_V_'), ((6, 9), '_Sp_')]
[((5, 8), '_T_'), ((6, 9), '_S_')]
[((5, 8), '_T_'), ((6, 9), '_V_')]
[((5, 8), '_T_'), ((6, 9), '_T_')]
[((5, 8), '_T_'), ((6, 9), '_Vp_')]
[((5, 8), '_T_'), ((6, 9), '_Sp_')]
[((5, 8), '_Vp_'), ((6, 9), '_S_')]
[((5, 8), '_Vp_'), ((6, 9), '_V_')]
[((5, 8), '_Vp_'), ((6, 9), '_T_')]
[((5, 8), '_Vp_'), ((6, 9), '_Vp_')]
[((5, 8), '_Vp_'), ((6, 9), '_Sp_')]
[((5, 8), '_Sp_'), ((6, 9), '_S_')]
[((5, 8), '_Sp_'), ((6, 9), '_V_')]
[((5, 8), '_Sp_'), ((6, 9), '_T_')]
[((5, 8), '_Sp_'), ((6, 9), '_Vp_')]
[((5, 8), '_Sp_'), ((6, 9), '_Sp_')]
[((5, 9), '_S_'), ((6, 8), '_S_')]
[((5, 9), '_S_'), ((6, 8), '_V_')

In [114]:
derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'D', 'P', 'P']
bilinear_contraction_list = [((5, 8), '_T_'), ((6, 9), '_S_')]
term_object = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
print(term_object.get_field_symbols())
print(term_object.get_spinor_contractions())

['D', 'F', 'D', 'D', 'D', 'Pb', '_T_', 'Pb', '_S_', 'D', 'P', 'P']
[(5, 6), (6, 10), (7, 8), (8, 11)]


In [115]:
derivative_assignment = ['D', 'F', 'D', 'D', 'D', 'Pb', 'Pb', 'D', 'P', 'P']
spinor_contracted_term = convert_to_term_object(derivative_assignment, bilinear_contraction_list)

#helper function for scalar_lorentz_contractions_from_spinor_contraction(spinor_contracted_term) below
def generate_pair_partitions(input_list):
    #EXPLANATION
    #generates all ways of partitioning the input list into pairs. if number of elements in list is odd,
    #generates all ways of partitioning the input list into pairs with one left over. 
    
    #recursion base case 1
    if len(input_list)==0:
        #print('BASE CASE, RETURN []')
        return [[]]
    elif len(input_list)%2==0:
        partition_list = [] #list of lists for collecting different partitions into pairs
        #input_list_copy = input_list.copy()
        #print('INPUT_LIST: ' + str(input_list))
        item1 = input_list[0]
        for j in range(1, len(input_list)):
            #print('i: ' + str(i))
            #print('j: ' + str(j))
            item2 = input_list[j]
            pair = (item1, item2)
            input_list_new = [input_list[k] for k in range(len(input_list)) if k not in [0,j]]
            #print('INPUT_LIST_NEW: ' + str(input_list_new))
            #print('INPUT_LIST_NEW, Post deletions: ' + str(input_list_new))
            for partition in generate_pair_partitions(input_list_new):
                partition_new = [pair] + partition
                partition_list.append(partition_new)
        return partition_list
    elif len(input_list)%2!=0:
        partition_list = []
        for i in range(len(input_list)):
            single = input_list[i]
            input_list_copy = input_list.copy()
            del input_list_copy[i]
            for partition in generate_pair_partitions(input_list_copy):
                #partition_new = [single] + partition #includes single uncontracte index in list
                #partition_list.append(partition_new)
                partition_list.append(partition)
        return partition_list         
            

In [116]:
input_list = [1,2,3,4,5,6,7]
generate_pair_partitions(input_list)

[[(2, 3), (4, 5), (6, 7)],
 [(2, 3), (4, 6), (5, 7)],
 [(2, 3), (4, 7), (5, 6)],
 [(2, 4), (3, 5), (6, 7)],
 [(2, 4), (3, 6), (5, 7)],
 [(2, 4), (3, 7), (5, 6)],
 [(2, 5), (3, 4), (6, 7)],
 [(2, 5), (3, 6), (4, 7)],
 [(2, 5), (3, 7), (4, 6)],
 [(2, 6), (3, 4), (5, 7)],
 [(2, 6), (3, 5), (4, 7)],
 [(2, 6), (3, 7), (4, 5)],
 [(2, 7), (3, 4), (5, 6)],
 [(2, 7), (3, 5), (4, 6)],
 [(2, 7), (3, 6), (4, 5)],
 [(1, 3), (4, 5), (6, 7)],
 [(1, 3), (4, 6), (5, 7)],
 [(1, 3), (4, 7), (5, 6)],
 [(1, 4), (3, 5), (6, 7)],
 [(1, 4), (3, 6), (5, 7)],
 [(1, 4), (3, 7), (5, 6)],
 [(1, 5), (3, 4), (6, 7)],
 [(1, 5), (3, 6), (4, 7)],
 [(1, 5), (3, 7), (4, 6)],
 [(1, 6), (3, 4), (5, 7)],
 [(1, 6), (3, 5), (4, 7)],
 [(1, 6), (3, 7), (4, 5)],
 [(1, 7), (3, 4), (5, 6)],
 [(1, 7), (3, 5), (4, 6)],
 [(1, 7), (3, 6), (4, 5)],
 [(1, 2), (4, 5), (6, 7)],
 [(1, 2), (4, 6), (5, 7)],
 [(1, 2), (4, 7), (5, 6)],
 [(1, 4), (2, 5), (6, 7)],
 [(1, 4), (2, 6), (5, 7)],
 [(1, 4), (2, 7), (5, 6)],
 [(1, 5), (2, 4), (6, 7)],
 

In [117]:
def scalar_lorentz_contractions_from_spinor_contraction(spinor_contracted_term):
    
    #make a list of positions of fields with one or more free Lorentz indices. positions of fields with more than
    #one lorentz index occur in the list as many times as there are indicies.
    field_list = spinor_contracted_term.get_field_list()
    spinor_contractions = spinor_contracted_term.get_spinor_contractions()
    lorentz_field_positions = [] #collect positions of fields associated with each lorentz index in spinor_contracted_term
    for i in range(len(field_list)):
        lorentz_rank = field_list[i].get_lorentz_rank()
        lorentz_field_positions += [i for j in range(lorentz_rank)]
    
    #generate all groupings of lorentz indices into pairs
    lorentz_contractions_list = generate_pair_partitions(lorentz_field_positions)
    
    #create new term for each item in lorentz_contractions; assemble terms into a list
    fully_contracted_list = []
    for lorentz_contractions in lorentz_contractions_list:
        new_term = term(field_list, lorentz_contractions, spinor_contractions)
        fully_contracted_list.append(new_term)
    
    return fully_contracted_list

In [125]:
derivative_assignment = ['F','Pb','Pb','Pb','D','D','P','D','P','D', 'D','P']
bilinear_contraction_list = [((1, 6), '_V_'), ((2, 8), '_V_'), ((3, 10), '_T_')]
spinor_contracted_term = convert_to_term_object(derivative_assignment, bilinear_contraction_list)
term_list = scalar_lorentz_contractions_from_spinor_contraction(spinor_contracted_term)

#get field positions of lorentz indices
field_list = term_list[0].get_field_list()
lorentz_field_positions = []
for i in range(len(field_list)):
    lorentz_rank = field_list[i].get_lorentz_rank()
    lorentz_field_positions += [i for j in range(lorentz_rank)]
    
for item in scalar_lorentz_contractions_from_spinor_contraction(spinor_contracted_term):
    print('')
    print(item.get_field_symbols())
    print(item.get_spinor_contractions())
    print(item.get_lorentz_contractions())
    print(lorentz_field_positions)
    
    
    


['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 7), (8, 10), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 7), (8, 12), (10, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 7), (8, 13), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 6), (6, 8), (7, 10), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), 

[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (2, 8), (6, 12), (6, 13), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (2, 8), (6, 13), (6, 7), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (2, 8), (6, 13), (6, 10), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (2, 8), (6, 13), (6, 12), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (2, 10), (6, 6), (7, 8), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 

[(0, 7), (2, 4), (6, 13), (6, 8), (10, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (2, 4), (6, 13), (6, 10), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (2, 4), (6, 13), (6, 12), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (2, 6), (4, 6), (8, 10), (12, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (2, 6), (4, 6), (8, 12), (10, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', '

[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 10), (2, 13), (4, 7), (6, 12), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 10), (2, 13), (4, 8), (6, 6), (7, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 10), (2, 13), (4, 8), (6, 7), (6, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 10), (2, 13), (4, 8), (6, 12), (6, 7)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 10), (2, 13), (4, 12), (6, 6), (7, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 


['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 12), (6, 8), (6, 7), (10, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 12), (6, 8), (6, 10), (7, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 12), (6, 8), (6, 13), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 2), (4, 12), (6, 10), (6, 7), (8, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), 

...

[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 6), (4, 6), (8, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 6), (4, 8), (6, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 6), (4, 13), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 10), (2, 8), (4, 6), (6, 13)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), 

[(0, 13), (0, 7), (2, 4), (6, 8), (6, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (0, 7), (2, 4), (6, 10), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (0, 7), (2, 6), (4, 6), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (0, 7), (2, 6), (4, 8), (6, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 13), (0, 7), (2, 6), (4, 10), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', '

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 7), (2, 8), (6, 10), (6, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 7), (2, 8), (6, 12), (6, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 7), (2, 10), (6, 6), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 4), (0, 7), (2, 10), (6, 8), (6, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 1

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 8), (4, 12), (6, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 10), (4, 6), (8, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 10), (4, 8), (6, 12)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 7), (0, 6), (2, 10), (4, 12), (6, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 1

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 12), (0, 6), (2, 4), (6, 10), (7, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 12), (0, 6), (2, 6), (4, 7), (8, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 12), (0, 6), (2, 6), (4, 8), (7, 10)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 13)]
[(0, 12), (0, 6), (2, 6), (4, 10), (7, 8)]
[0, 0, 2, 4, 6, 6, 7, 8, 10, 12, 13]

['F', 'Pb', '_V_', 'Pb', '_V_', 'Pb', '_T_', 'D', 'D', 'P', 'D', 'P', 'D', 'D', 'P']
[(1, 2), (2, 9), (3, 4), (4, 11), (5, 6), (6, 1

In [65]:
def scalar_lorentz_contractions_from_spinor_contraction(spinor_contracted_term):
    
    return fully_contracted_scalars


def generate_lorentz_scalars(massDim): #terms with one free Lorentz index and all other indices 
    #contracted
    return lorentz_scalar_list

def generate_lorentz_vectors(massDim): #terms with one free Lorentz index and all other indices 
    #contracted
    return lorentz_vector_list

### Remove Redundant Lorentz Contractions

In [66]:
def terms_equiv(term_1, term_2):
    return equiv

def reduce_fully_contracted_terms(fully_contracted_list):
    return fully_contracted_list_reduced

def reduce_contracted_lorentz_vectors(lorentz_vector_list):
    return fully_contracted_list_reduced


# Fierz Identities

# Integration by Parts

# Equations of Motion